<a href="https://colab.research.google.com/github/dine1717/EVA6/blob/session4/EVA6_S4_MNIST_10KParam_20Epochs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Fri May 28 16:20:43 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    25W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import ReduceLROnPlateau

In [ ]:
!pip install torchsummary

In [ ]:
use_cuda = torch.cuda.is_available()
torch.manual_seed(12)
batch_size = 32

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.RandomRotation((-7.0, 7.0), fill=(1,)),
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw




Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw




Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw

Processing...
Done!


/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:502: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:143.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [ ]:
from tqdm import tqdm

train_losses = []
test_losses = []
train_acc = []
test_acc = []

def train(model, device, train_loader, optimizer, epoch):
  model.train()
  pbar = tqdm(train_loader)
  correct = 0
  processed = 0
  for batch_idx, (data, target) in enumerate(pbar):
    # get samples
    data, target = data.to(device), target.to(device)

    # Init
    optimizer.zero_grad()
    # In PyTorch, we need to set the gradients to zero before starting to do backpropragation because PyTorch accumulates the gradients on subsequent backward passes. 
    # Because of this, when you start your training loop, ideally you should zero out the gradients so that you do the parameter update correctly.

    # Predict
    y_pred = model(data)

    # Calculate loss
    loss = F.nll_loss(y_pred, target)
    train_losses.append(loss)

    # Backpropagation
    loss.backward()
    optimizer.step()

    # Update pbar-tqdm
    
    pred = y_pred.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
    correct += pred.eq(target.view_as(pred)).sum().item()
    processed += len(data)

    pbar.set_description(desc= f'Loss={loss.item()} Batch_id={batch_idx} Accuracy={100*correct/processed:0.2f}')
    train_acc.append(100*correct/processed)


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    #scheduler = ReduceLROnPlateau(optimizer, factor=0.9)
    #scheduler.step(test_loss)
    test_losses.append(test_loss)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    
    test_acc.append(100. * correct / len(test_loader.dataset))

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1=nn.Sequential(nn.Conv2d(1,16,3,padding=0,bias=False),        #conv1  1 input , 16 0utput and 3x3 kernel
                                 nn.ReLU(),                                     #RF:3
                                 nn.BatchNorm2d(16),                            #o/p:26
                                 nn.Dropout2d(0.04))
        
        
        self.conv2=nn.Sequential(nn.Conv2d(16,24,3,padding=0,bias=False),       #conv2  16 input , 32 0utput and 3x3 kernel
                                 nn.ReLU(),                                     #RF:5
                                 nn.BatchNorm2d(24),                            #o/p:24
                                 nn.Dropout2d(0.04))
        



        # used 1x1 kernel 
        self.conv3=nn.Sequential(nn.Conv2d(24,12,1,bias=False))                  #conv3  32 input , 8 0utput and 1x1 kernel 
                                                                                #RF:5
                                                                                #o/p:24
        


        self.maxpool=nn.MaxPool2d(2,2)                                          # maxpooling 2x2 kernel and stride 2
                                                                                #RF:6
                                                                                #o/p:12


     

        self.conv4=nn.Sequential(nn.Conv2d(12,16,3,padding=0,bias=False),        #conv4  8 input , 14 0utput and 3x3 kernel
                                 nn.ReLU(),                                     #RF:10
                                 nn.BatchNorm2d(16),                            #o/p:10
                                 nn.Dropout2d(0.04))
                                                                                
        
        self.conv5=nn.Sequential(nn.Conv2d(16,24,3,padding=0,bias=False),       #conv5  14 input , 16 0utput and 3x3 kernel
                                 nn.ReLU(),                                     #RF:14
                                 nn.BatchNorm2d(24),                            #o/p:8
                                 nn.Dropout2d(0.04))                                                
                                                                                
                                                                
        
        #self.maxpool=nn.MaxPool2d(2,2)
                                                               
        self.conv6=nn.Sequential(nn.Conv2d(24,20,1,padding=0,bias=False),
                                 nn.ReLU(),
                                 nn.BatchNorm2d(20),
                                 nn.Dropout2d(0.04)      )       #conv6  16 input , 10 0utput and 1x1 kernel
                                                                                #RF:14
                                                                                #o/p:8
                                                                                

        #self.conv7=nn.Sequential(nn.Conv2d(16,10,3,padding=0,bias=False),
                                 #nn.ReLU(),                                     #conv6  16 input , 10 0utput and 3x3 kernel
                                 #nn.BatchNorm2d(16),                            #RF:18
                                 #nn.Dropout2d(0.04))                            #o/p:6           
                                                                                
        
        self.GAP=nn.AvgPool2d(8)                                                # Gobal Average Pooling 
                                                                                #RF:28
                                                                                #o/p:1
        #self.flatten_gap = nn.Sequential(nn.AvgPool2d(8), nn.Flatten())
        #self.final_linear = nn.Sequential(nn.Linear(20,10))
                                                                                

        self.conv8 = nn.Sequential(nn.Conv2d(20,10,1, padding=0, bias=False))   #RF:28
                                                                                #O/P:1                                           
                                                                                


             
    def forward(self,x):
        x=self.conv1(x)
        x=self.conv2(x)
        #x=self.maxpool(x)
        x=self.conv3(x)
        x=self.maxpool(x)
        x=self.conv4(x)
        x=self.conv5(x)
        #x=self.maxpool(x)
        x=self.conv6(x)
        #x=self.conv7(x)
        x = self.GAP(x)
        #x=self.flatten_gap(x)
        #x=self.final_linear(x)
        x=self.conv8(x)
        x=x.view(-1,10)
        return F.log_softmax(x)

#!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 26, 26]             144
              ReLU-2           [-1, 16, 26, 26]               0
       BatchNorm2d-3           [-1, 16, 26, 26]              32
         Dropout2d-4           [-1, 16, 26, 26]               0
            Conv2d-5           [-1, 24, 24, 24]           3,456
              ReLU-6           [-1, 24, 24, 24]               0
       BatchNorm2d-7           [-1, 24, 24, 24]              48
         Dropout2d-8           [-1, 24, 24, 24]               0
            Conv2d-9           [-1, 12, 24, 24]             288
        MaxPool2d-10           [-1, 12, 12, 12]               0
           Conv2d-11           [-1, 16, 10, 10]           1,728
             ReLU-12           [-1, 16, 10, 10]               0
      BatchNorm2d-13           [-1, 16, 10, 10]              32
        Dropout2d-14           [-1, 16,

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:90: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [ ]:
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 20):
    print("epoches:",epoch)
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

epoches: 1


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:90: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.




Test set: Average loss: 0.0637, Accuracy: 9817/10000 (98.17%)

epoches: 2




Test set: Average loss: 0.0449, Accuracy: 9863/10000 (98.63%)

epoches: 3




Test set: Average loss: 0.0337, Accuracy: 9890/10000 (98.90%)

epoches: 4


...



Test set: Average loss: 0.0231, Accuracy: 9922/10000 (99.22%)

epoches: 16




Test set: Average loss: 0.0227, Accuracy: 9923/10000 (99.23%)

epoches: 17




Test set: Average loss: 0.0194, Accuracy: 9926/10000 (99.26%)

epoches: 18




Test set: Average loss: 0.0217, Accuracy: 9929/10000 (99.29%)

epoches: 19




Test set: Average loss: 0.0177, Accuracy: 9937/10000 (99.37%)



In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1=nn.Sequential(nn.Conv2d(1,16,3,padding=0,bias=False),        #conv1  1 input , 16 0utput and 3x3 kernel
                                 nn.ReLU(),                                     #RF:3
                                nn.BatchNorm2d(16),                            #o/p:26
                                 nn.Dropout2d(0.04))
        
        
        self.conv2=nn.Sequential(nn.Conv2d(16,20,3,padding=0,bias=False),       #conv2  16 input , 32 0utput and 3x3 kernel
                                 nn.ReLU(),                                     #RF:5
                                 nn.BatchNorm2d(20),                            #o/p:24
                                 nn.Dropout2d(0.04))
        



        # used 1x1 kernel 
        self.conv3=nn.Sequential(nn.Conv2d(20,12,1,bias=False))                  #conv3  32 input , 8 0utput and 1x1 kernel 
                                                                                #RF:5
                                                                                #o/p:24
        


        self.maxpool=nn.MaxPool2d(2,2)                                          # maxpooling 2x2 kernel and stride 2
                                                                                #RF:6
                                                                                #o/p:12


     

        self.conv4=nn.Sequential(nn.Conv2d(12,16,3,padding=0,bias=False),        #conv4  8 input , 14 0utput and 3x3 kernel
                                 nn.ReLU(),                                     #RF:10
                                 nn.BatchNorm2d(16),                            #o/p:10
                                 nn.Dropout2d(0.04))
                                                                                
        
        self.conv5=nn.Sequential(nn.Conv2d(16,20,3,padding=0,bias=False),       #conv5  14 input , 16 0utput and 3x3 kernel
                                 nn.ReLU(),                                     #RF:14
                                 nn.BatchNorm2d(20),                            #o/p:8
                                 nn.Dropout2d(0.04))                                                
                                                                                
                                                                
        
        #self.maxpool=nn.MaxPool2d(2,2)
                                                               
        self.conv6=nn.Sequential(nn.Conv2d(20,10,1,padding=0,bias=False))       #conv6  16 input , 10 0utput and 3x3 kernel
                                                                                #RF:14
                                                                                #o/p:8
                                                                                

        self.conv7=nn.Sequential(nn.Conv2d(10,16,3,padding=0,bias=False),
                                 nn.ReLU(),                                     #conv6  10 input , 16 0utput and 3x3 kernel
                                 nn.BatchNorm2d(16),                            #RF:18
                                 nn.Dropout2d(0.04))                            #o/p:6           
                                                                                
        
        self.GAP=nn.AvgPool2d(6)                                                # Gobal Average Pooling 
                                                                                #RF:28
                                                                                #o/p:1

                                                                                

        self.conv8 = nn.Sequential(nn.Conv2d(16,10,1, padding=0, bias=False))   #RF:28
                                                                                #O/P:1                                           
                                                                                


             
    def forward(self,x):
        x=self.conv1(x)
        x=self.conv2(x)
        #x=self.maxpool(x)
        x=self.conv3(x)
        x=self.maxpool(x)
        x=self.conv4(x)
        x=self.conv5(x)
        #x=self.maxpool(x)
        x=self.conv6(x)
        x=self.conv7(x)
        x=self.GAP(x)
        x=self.conv8(x)
        x=x.view(-1,10)
        return F.log_softmax(x)


#!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 26, 26]             144
              ReLU-2           [-1, 16, 26, 26]               0
       BatchNorm2d-3           [-1, 16, 26, 26]              32
         Dropout2d-4           [-1, 16, 26, 26]               0
            Conv2d-5           [-1, 20, 24, 24]           2,880
              ReLU-6           [-1, 20, 24, 24]               0
       BatchNorm2d-7           [-1, 20, 24, 24]              40
         Dropout2d-8           [-1, 20, 24, 24]               0
            Conv2d-9           [-1, 12, 24, 24]             240
        MaxPool2d-10           [-1, 12, 12, 12]               0
           Conv2d-11           [-1, 16, 10, 10]           1,728
             ReLU-12           [-1, 16, 10, 10]               0
      BatchNorm2d-13           [-1, 16, 10, 10]              32
        Dropout2d-14           [-1, 16,

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:84: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [ ]:
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 20):
    print("epoches:",epoch)
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/1875 [00:00<?, ?it/s]

epoches: 1


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:84: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
Loss=0.1682090163230896 Batch_id=1874 Accuracy=91.70: 100%|██████████| 1875/1875 [00:25<00:00, 73.02it/s]
  0%|          | 0/1875 [00:00<?, ?it/s]


Test set: Average loss: 0.0532, Accuracy: 9834/10000 (98.34%)

epoches: 2


Loss=0.04723441228270531 Batch_id=1874 Accuracy=97.19: 100%|██████████| 1875/1875 [00:25<00:00, 72.21it/s]
  0%|          | 0/1875 [00:00<?, ?it/s]

...


Test set: Average loss: 0.0188, Accuracy: 9935/10000 (99.35%)

epoches: 18


Loss=0.012451915070414543 Batch_id=1874 Accuracy=98.75: 100%|██████████| 1875/1875 [00:26<00:00, 71.65it/s]
  0%|          | 0/1875 [00:00<?, ?it/s]


Test set: Average loss: 0.0196, Accuracy: 9938/10000 (99.38%)

epoches: 19


Loss=0.006165508180856705 Batch_id=1874 Accuracy=98.80: 100%|██████████| 1875/1875 [00:26<00:00, 70.91it/s]



Test set: Average loss: 0.0213, Accuracy: 9930/10000 (99.30%)

